In [11]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import chain
from sklearn.model_selection import train_test_split

from qiskit import BasicAer, QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.circuit.library import QFT

seed = 10599
aqua_globals.random_seed = seed

In [7]:
dataset = np.load('../data/images.npy')[:10]
dataset_labels = np.load('../data/labels.npy')[:10]
print('No of entries in dataset {}'.format(len(dataset)))

No of entries in dataset 10


In [8]:
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_labels, test_size=0.33, random_state=seed)
print('shape of X_train {}'.format(X_train.shape))
print('shape of y_train {}'.format(y_train.shape))
print('shape of X_test {}'.format(X_test.shape))
print('shape of y_test {}'.format(y_test.shape))
print('Size of train set {}'.format(len(X_train)))
print('Size of test set {}'.format(len(X_test)))

shape of X_train (6, 28, 28)
shape of y_train (6,)
shape of X_test (4, 28, 28)
shape of y_test (4,)
Size of train set 6
Size of test set 4


In [17]:
training_input = {'0': [], '1': []}
test_input = {'0': [], '1': []}

for i in range(len(y_train)):
    if y_train[i] == True:
        training_input['1'].append(X_train[i])
    else:
        training_input['0'].append(X_train[i])
# print(training_input)
# print(X_train)
# print(y_train)
for i in range(len(y_test)):
    if y_test[i] == True:
        test_input['1'].append(X_test[i])
    else:
        test_input['0'].append(X_test[i])

In [18]:
def encode_img(image, register_num=10):
    # img = Image.open(image, 'r')
    # img = img.convert("L")  # grayscale
    #
    # pix_val = list(img.getdata())
    img = list(chain(*image))
    pix_val = img
    # arr_pix_val = 255*255*pix_val
    # print(pix_val)

    # normalize
    pix_norm = np.linalg.norm(pix_val)
    # print(pix_norm)
    pix_val = np.array(pix_val)
    arr_norm = pix_val/pix_norm
    arr_norm = arr_norm.tolist()

    # Encode onto the quantum register
    qc = QuantumCircuit(register_num)
    # test = arr_norm.append(np.zeros(2**10-arr_norm.shape))
    test = arr_norm + np.zeros(2**register_num-784).tolist()
    qc.initialize(test, qc.qubits)
    return qc

# apply qft


def apply_qft(circuit, register_num):
    circuit.append(
        QFT(register_num, do_swaps=False).to_gate(), circuit.qubits)
    return circuit

In [21]:
# feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, entanglement='linear')
enc_image = encode_img(X_train[0])
qsvm = QSVM(enc_image, training_input, test_input)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

result = qsvm.run(quantum_instance)

print(f'Testing success ratio: {result["testing_accuracy"]}')
print()
# print('Prediction from datapoints set:')
# print(f'  ground truth: {map_label_to_class_name(datapoints[1], qsvm.label_to_class)}')
# print(f'  prediction:   {result["predicted_classes"]}')
# predicted_labels = result["predicted_labels"]
# print(f'  success rate: {100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)}%')

/home/varshaneya/iquhack/ionqenv/lib/python3.6/site-packages/requests_ntlm/requests_ntlm.py:5: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography (40.0) will be the last to support Python 3.6.
  from cryptography import x509


CircuitError: "Invalid param type <class 'complex'> for gate initialize_dg."